### Applying Naive Bayes Algorithm on Amazon Alexa Reviews Dataset

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from collections import Counter
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,classification_report,f1_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV


In [3]:
data = pd.read_csv("C:/Users/prady/Downloads/AI/tp/amazon_alexa_reviews.csv")
#importing the csv file

In [4]:
data.shape

(3150, 5)

In [5]:
data.head(10)

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
5,5,31-Jul-18,Heather Gray Fabric,I received the echo as a gift. I needed anothe...,1
6,3,31-Jul-18,Sandstone Fabric,"Without having a cellphone, I cannot use many ...",1
7,5,31-Jul-18,Charcoal Fabric,I think this is the 5th one I've purchased. I'...,1
8,5,30-Jul-18,Heather Gray Fabric,looks great,1
9,5,30-Jul-18,Heather Gray Fabric,Love it! I’ve listened to songs I haven’t hear...,1


In [6]:
# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = data['rating']
positiveNegative = actualScore.map(partition) 
data['rating'] = positiveNegative

In [7]:
data.head()

,rating,date,variation,verified_reviews,feedback
0,positive,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,positive,31-Jul-18,Charcoal Fabric,Loved it!,1
2,positive,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,positive,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,positive,31-Jul-18,Charcoal Fabric,Music,1


In [8]:
# find sentences containing HTML tags
import re
i=0;
for sent in data['verified_reviews'].values:
    if (len(re.findall('<.*?>', sent))):
        print(i)
        print(sent)
        break;
    i += 1;

In [9]:
stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)
print('************************************')
print(sno.stem('nice'))

{'am', 'theirs', 'each', 'she', "don't", 't', 's', 'ma', 'don', 're', 'until', "couldn't", 'has', 'for', "isn't", 'why', 'how', 'hadn', "didn't", 'only', 'both', 'or', "won't", 'out', 'under', 'couldn', 'all', 'shouldn', 'into', 'doesn', 'a', 'once', 'o', 'over', 'any', 'they', 'your', 'he', "she's", 'can', 'which', 'against', 'weren', 'off', 'm', 'did', 'me', 'as', 'then', "doesn't", 'isn', 'most', "wasn't", 'own', 'now', 'itself', 'such', "weren't", "aren't", 'was', 'before', 'that', 'is', 'been', "should've", 'aren', 'my', 'few', 'about', 'their', 'those', 'having', 'we', 'no', 'll', 'too', 'be', 'other', 'below', 'on', 'by', 'his', "you'd", "you're", 'not', 'the', 'should', 'nor', 'them', 'hasn', 'didn', 'of', 'at', 'during', 'have', 'through', 'here', 've', 'will', 'haven', 'had', 'to', "wouldn't", 'who', "that'll", 'an', 'than', 'very', 'wasn', 'mustn', 'it', 'hers', 'and', 'while', 'so', 'her', 'if', 'this', 'were', 'ours', 'up', 'y', "haven't", 'down', 'mightn', 'some', 'd', 'o

In [11]:
#Code for implementing step-by-step the checks mentioned in the pre-processing phase

i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in data['verified_reviews'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (data['rating'].values)[i] == 'positive': 
                        all_positive_words.append(s) #list of all words used to describe positive reviews
                    if(data['rating'].values)[i] == 'negative':
                        all_negative_words.append(s) #list of all words used to describe negative reviews reviews
                else:
                    continue
            else:
                continue 
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    
    final_string.append(str1)
    i+=1

In [13]:
data['CleanedText']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review 
data['CleanedText']=data['CleanedText'].str.decode("utf-8")

In [14]:
data.head(10)

,rating,date,variation,verified_reviews,feedback,CleanedText
0,positive,31-Jul-18,Charcoal Fabric,Love my Echo!,1,love echo
1,positive,31-Jul-18,Charcoal Fabric,Loved it!,1,love
2,positive,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1,sometim play game answer question correct alex...
3,positive,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1,lot fun thing old learn dinosaur control light...
4,positive,31-Jul-18,Charcoal Fabric,Music,1,music
5,positive,31-Jul-18,Heather Gray Fabric,I received the echo as a gift. I needed anothe...,1,receiv echo gift need anoth bluetooth someth p...
6,positive,31-Jul-18,Sandstone Fabric,"Without having a cellphone, I cannot use many ...",1,without cellphon cannot use mani featur ipad s...
7,positive,31-Jul-18,Charcoal Fabric,I think this is the 5th one I've purchased. I'...,1,think one ive purchas work get one everi room ...
8,positive,30-Jul-18,Heather Gray Fabric,looks great,1,look great
9,positive,30-Jul-18,Heather Gray Fabric,Love it! I’ve listened to songs I haven’t hear...,1,love listen song heard sinc childhood get news...


In [17]:
a=data["CleanedText"]
b=data["rating"]

### Splitting of data into Train and Test 
To hide the Test data and perform methods on Train data, we split the data in the (70-30) ratio.

In [19]:
X_train=a.iloc[:2205,]
X_test=a.iloc[2205:3150,]

In [26]:
y_train=b.iloc[:2205,]
y_test=b.iloc[2205:3150,]

# Bag of Words (BoW)

In [27]:

count_vect = CountVectorizer() 
x_train_vect_bow = count_vect.fit_transform(X_train.values.astype('U'))
x_test_vect_bow = count_vect.transform(X_test.values.astype('U'))



In [28]:
nb = MultinomialNB()
alpha=[10**x for x in range(-3,3)]
params={'alpha':alpha}
clf = GridSearchCV(nb,param_grid=params,cv=3)

In [29]:
clf.fit(x_train_vect_bow, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [30]:
clf.best_params_

{'alpha': 0.1}

In [31]:
clf.best_estimator_

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [33]:
nb = MultinomialNB(alpha=1)
        # fitting the model on crossvalidation train
nb.fit(x_train_vect_bow, y_train)

        
pred = nb.predict(x_test_vect_bow)



In [34]:
print('Accuracy with alpha =',clf.best_params_,' is ' ,np.round((accuracy_score(y_test, pred)*100)))

Accuracy with alpha = {'alpha': 0.1}  is  94.0


In [35]:
print ('\n Classification report with alpha = ',clf.best_params_,' is \n' , classification_report(y_test,pred))


 Classification report with alpha =  {'alpha': 0.1}  is 
               precision    recall  f1-score   support

    negative       0.76      0.19      0.30        69
    positive       0.94      1.00      0.97       876

   micro avg       0.94      0.94      0.94       945
   macro avg       0.85      0.59      0.63       945
weighted avg       0.93      0.94      0.92       945



### Conclusion:

I took Amazon ALexa reviews dataset from Kaggle and preprocessed the data. Then I used Bag Of Words(BOW) vectorizer after splitting the data into train and test datasets and implemented BOW on train data. Then I've encorporated Naive Bayes algorithm by choosing the best Alpha value from Grid Search CV and performed it on test data to predict the rating of a review from test dataset. After fetching the values, I've checked for accuracy which is found to be 94%.